In [1]:
import requests
import json
import xmltodict
import os
import time
import numpy as np
import pandas as pd

In [2]:
# Function to perform API call and return Python Dictionary containing data
def api_call(db, protein):
    """
    Performs API call to NCBI for specified database and protein
    Parameters:
    -----------
    db : str
        NCBI database to search for, i.e. 'protein', 'nuccore', etc.
    protein : str
        Protein accession number to search for
    """
    # Create url for API call
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=" + db + "&id=" + protein
    
    # Define tempory XML and JSON files
    xml_file = "data.xml"
    json_file = "data.json"

    # Perform API call
    resp = requests.get(url)
    
    # Save XML result to temporary file
    with open(xml_file, "wb") as f:
        f.write(resp.content)

    # Read XML file and convert to OrderedDict using xmltodict
    with open(xml_file, "r") as f:
        data_dict = xmltodict.parse(f.read())
    
    # Write OrderedDict to JSON file
    json_data = json.dumps(data_dict)
    with open(json_file, "w") as f:
        f.write(json_data)
    
    # Read in JSON file into regular Python dictionary
    with open(json_file, "r") as f:
        data = json.load(f)
    
    # Clean up temporary files
    os.remove(xml_file)
    os.remove(json_file)

    return data

In [22]:
# Helper function to get a protein sequence and its length
def get_sequence_and_count(protein_df, protein):
    protein_indeces = protein_df.index
    idx_array = protein_indeces[protein_df["accession_num"] == protein]
    idx = idx_array[0]
    seq_raw = protein_df[protein_df["accession_num"] == protein]["Sequence"][idx]
    seq_array = seq_raw.split("\n")
    seq = ""
    for i in range(1, len(seq_array)):
        seq += seq_array[i]
    return (len(seq), seq_raw)

In [4]:
# Read protein accession numbers from proteins.txt
infile = os.path.join("Resources", "proteins.csv")
protein_df = pd.read_csv(infile)
protein_df

,accession_num,aa_cnt
0,NP_001116538,776
1,Q5YCV9,776
2,XP_012352933,732
3,XP_002800600,776
4,XP_003913279,776
...,...,...
83,XP_008116759,1176
84,NP_001119982,1164
85,XP_006634278,629
86,XP_006003167,1753


In [5]:
# Perform API Call to NCBI to get GIDs for each Protein
gids = []
err_proteins = []  # list to hold proteins not found
protein_proteins = []  # list to hold proteins found in protein db
nuccore_proteins = []  # list to hold proteins found in nuccore db

# Be nice: no more than 3 calls per second -> every three calls wait 1 second
for i in range(len(protein_df)):
    print("Protein %s of %s" % (i + 1, len(protein_df)), end="\r")

    if (i % 3) == 0:
        time.sleep(1)

    # Search protein db
    protein = protein_df.iloc[i]["accession_num"]
    result = api_call("protein", protein)
    try:
        gid = result["eSummaryResult"]["DocSum"]["Id"]
        protein_proteins.append(protein)
        gids.append(gid)
    except KeyError:
        # If error, search nuccore
        try:
            result = api_call("nuccore", protein)
            gid = result["eSummaryResult"]["DocSum"]["Id"]
            nuccore_proteins.append(protein)
            gids.append(gid)
        # If still error, add NaN
        except KeyError:
            err_proteins.append(protein)
            gids.append(np.NaN)
gids[0:5]

['294862258', '59798492', '821025767', '297273333', '1777289710']

In [6]:
print("Total Number of Proteins in Input File:", len(protein_df))
print("\n")
print("Number of Proteins From db=protein:", len(protein_proteins))
print ("Proteins From db=nuccore (%s):" % len(nuccore_proteins))
for protein in nuccore_proteins:
    print(protein)
print("\n")
print("Proteins Not Found (%s):" % len(err_proteins))
for protein in err_proteins:
    print(protein)
print("\n")

Total Number of Proteins in Input File: 88


Number of Proteins From db=protein: 79
Proteins From db=nuccore (5):
GL477576
CT004140
BAHO01035973
KE993814
NW_003943621


Proteins Not Found (4):
scaffold11486
JL1528
scaffold43622
XP_01266736




In [7]:
# Add GIDs to proteins_df
protein_df["GID"] = gids
protein_df.head()

,accession_num,aa_cnt,GID
0,NP_001116538,776,294862258
1,Q5YCV9,776,59798492
2,XP_012352933,732,821025767
3,XP_002800600,776,297273333
4,XP_003913279,776,1777289710


In [8]:
# Confirm missing GIDs consistent with API result:
is_NaN = protein_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
protein_NaN = protein_df[row_has_NaN]
protein_NaN

,accession_num,aa_cnt,GID
22,scaffold11486,761,NaN
31,JL1528,864,NaN
54,scaffold43622,1947,NaN
70,XP_01266736,1119,NaN


In [9]:
# Need to output sequences to FASTA (.faa) file for alignment before bassing to RAxML to generate
# Phylogenetic tree
# FASTA Format:
#>SEQUENCE_1
#...
#>SEQUENCE_2
#...
#>...

In [10]:
# Get Sequences from GID numbers and write to FASTA file:
cnt = 0
seqs = []
# Write sequence to FASTA file
for gid in gids:
    print("Protein %s of %s" % (cnt + 1, len(protein_df)), end="\r")
    if (cnt % 3) == 0:
        time.sleep(1)
    if gid is not np.NaN:
        gid_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=sequences&id=" + gid + "&rettype=fasta&retmode=text"
        gid_resp = requests.get(gid_url)
        seq_fasta = gid_resp.content.decode("utf-8")
        seqs.append(seq_fasta)

    else:
        seqs.append(np.NaN)
    cnt += 1

In [11]:
# Add sequences to protein_df
protein_df["Sequence"] = seqs
protein_df.head()

,accession_num,aa_cnt,GID,Sequence
0,NP_001116538,776,294862258,>NP_001116538.2 microtubule-associated protein...
1,Q5YCV9,776,59798492,>sp|Q5YCV9.4|TAU_HYLLA RecName: Full=Microtubu...
2,XP_012352933,732,821025767,>XP_012352933.1 PREDICTED: microtubule-associa...
3,XP_002800600,776,297273333,>XP_002800600.1 PREDICTED: microtubule-associa...
4,XP_003913279,776,1777289710,>XP_003913279.2 microtubule-associated protein...


In [12]:
# Check a sequence
print(get_sequence_and_count(protein_df, "NP_001116538"))

Sequence Count: 776
>NP_001116538.2 microtubule-associated protein tau isoform 6 [Homo sapiens]
MAEPRQEFEVMEDHAGTYGLGDRKDQGGYTMHQDQEGDTDAGLKESPLQTPTEDGSEEPGSETSDAKSTP
TAEDVTAPLVDEGAPGKQAAAQPHTEIPEGTTAEEAGIGDTPSLEDEAAGHVTQEPESGKVVQEGFLREP
GPPGLSHQLMSGMPGAPLLPEGPREATRQPSGTGPEDTEGGRHAPELLKHQLLGDLHQEGPPLKGAGGKE
RPGSKEEVDEDRDVDESSPQDSPPSKASPAQDGRPPQTAAREATSIPGFPAEGAIPLPVDFLSKVSTEIP
ASEPDGPSVGRAKGQDAPLEFTFHVEITPNVQKEQAHSEEHLGRAAFPGAPGEGPEARGPSLGEDTKEAD
LPEPSEKQPAAAPRGKPVSRVPQLKARMVSKSKDGTGSDDKKAKTSTRSSAKTLKNRPCLSPKHPTPGSS
DPLIQPSSPAVCPEPPSSPKYVSSVTSRTGSSGAKEMKLKGADGKTKIATPRGAAPPGQKGQANATRIPA
KTPPAPKTPPSSATKQVQRRPPPAGPRSERGEPPKSGDRSGYSSPGSPGTPGSRSRTPSLPTPPTREPKK
VAVVRTPPKSPSSAKSRLQTAPVPMPDLKNVKSKIGSTENLKHQPGGGKVQIINKKLDLSNVQSKCGSKD
NIKHVPGGGSVQIVYKPVDLSKVTSKCGSLGNIHHKPGGGQVEVKSEKLDFKDRVQSKIGSLDNITHVPG
GGNKKIETHKLTFRENAKAKTDHGAEIVYKSPVVSGDTSPRHLSNVSSTGSIDMVDSPQLATLADEVSAS
LAKQGL




In [14]:
# Confirm consistent NaN's
is_NaN = protein_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
protein_NaN = protein_df[row_has_NaN]
protein_NaN

,accession_num,aa_cnt,GID,Sequence
22,scaffold11486,761,NaN,NaN
31,JL1528,864,NaN,NaN
54,scaffold43622,1947,NaN,NaN
70,XP_01266736,1119,NaN,NaN


In [23]:
# Example nuccore sequence
sea_lamprey = nuccore_proteins[0]  # GL477576
print(nuccore_proteins)
get_sequence_and_count(protein_df, sea_lamprey)
# nucleotide sequence (210407 nucleotides) for Petromyzon marinus (correct)
# Should have 846aa

['GL477576', 'CT004140', 'BAHO01035973', 'KE993814', 'NW_003943621']


(210407,
 '>GL477576.1 Petromyzon marinus unplaced genomic scaffold scaffold_1248, whole genome shotgun sequence\...\n\n')

In [24]:
hagfish = nuccore_proteins[1]  # CT004140
print(nuccore_proteins)
get_sequence_and_count(protein_df, hagfish)
# mRNA Sequence (837 nucleotides) for homo sapiens
# should be 243aa japanese inshore hagfish

['GL477576', 'CT004140', 'BAHO01035973', 'KE993814', 'NW_003943621']


(837,
 ">CT004140.1 CT004140 RZPD no.9017 Homo sapiens cDNA clone RZPDp9017M1121 5', mRNA sequence\nCACAGTGTGTGTGGAGCTGGACCTTCAGAACCTGGCCGTCAACACCATTCTCCAGTCTCACCATGTCAAC\nGACATGGAGGGGGCCGACAGCATGGGGATCAGCTTCGACGTGATCCAGCAGGCCTCCCTGGAGGACATGA\nTCACGGGTGGTCAGGCTGCCAGTGCCCCGGCCAGCTACGATGAGACAGCCCTGTGCTCTAACACCTTCAG\nGATCCTGAAGAGCATGGTCGTGGGCTGGGTGAAGGAGATCACCCAGTACCACAACATCTATGCAGACAAC\nCAGGTGATGCACTTCTACCGCTGGCTTCGGTCGCCATCCTCTCTGCTTCATGACCCTGCCCTGCACCGCA\nCACTCCACAACATGATGAAGAAGCTCTTCCTGCAGCTCATCGCTGAGTTCAAGCGCCTGGGGTCATCAGT\nCATCTACGCCAACTTCAACCGCATCATCCTCTGTACAAAGAAGCGCCGTGTGGAAGATGCCATCGCTTAC\nGTGGAGTACATCACCAGCAGCATCCATTCAAAGGAGACCTTCCATTCTCTGACAATTTCTTTCTCTCGAT\nGCTGGGAATTTCTTCTCTGGATGGATCCATCTAACTATGGCGGAATCAAAGGAAAAGTTTCATCTCGTAT\nTCACTGTGGACTGCAAGACTCCCAGAAAGCAGGGGGAGCAGAGGATGAGCAGGAAAATGAGGACGATGAG\nGAGGAAAGAGATGGGGANGAAGAAGGAAAAGGCGNAGGAATCCAACGNGGAGGATTTACTGGAAAACAAC\nTGGAACATTTTNGCAGTTTTGCCACAGGNANCCTCCTGNCANAACTACTTCCTCATGATTGNTTCAC\n\n")

In [25]:
coelacanth = nuccore_proteins[2]  # BAHO01035973
print(nuccore_proteins)
get_sequence_and_count(protein_df, coelacanth)
# nucleotide sequence (15919 nucleotides) for Latimeria chalumnae (correct)
# Should have 2118 aa

['GL477576', 'CT004140', 'BAHO01035973', 'KE993814', 'NW_003943621']


(15919,
 '>BAHO01035973.1 Latimeria chalumnae DNA, contig: contig2137.54, isolate: TCC041-004, whole genome shotgun sequence\nAAAAAAAAAATCAAATGACAACCTCACTGGTTAATTATTTTCAGCCTGGCTCAGTGCTGGGCAAAAAAAA\nAAAAAAAGAAAAGAAAACTGACGGCACTTAACAAGTGGAAAATAGAATATCTACAGGAAAGTAAGTTCTA\nAAGAACACAAAAGAAACCGTATAATCAATTACATATGCTCCAAATTTATTTATTTATTTTTTTAAATCAC\nGTTACATATCCACTTTAAGCTAATCAGTGCTGGCTGAAAATCATTATCTAGACCACACCTTTTGAGAAAA\nAAAAAAAAGTTTAGCATTTCTAGTTCACTAGTCATTGGCAAGTAACTTTTAAAATACAGTACCCTGGCTT\nCAACTGGACAATTTCAGGCCAGAAAATGATCAACAGAATATTCCACAACAGAAAGTGTTTGTTTAAAATG\nATTTTACCAGGGGGAAAAAACTCATTGAGATACAACATCTCGTTTTCAAAAGATACAACTATAATACATA\nAATATGATGATTTTTTTTTTTTTTATCCAAGTAGCGCAAACGCGCTGCCCTCTAGGAGGCTGGGGTTCAG\nTCTTTGCCATAGATGTCCCCAAGCCCCCCGGGTGGCTGAGGCTGAGATGCATTGCAGAAGCGGTACACTC\nAACCTCGGGGGAAGGAATCTGGCTCCTCCCCCTGTTTGAGCGCCCTCTAGTGGCAGACTCAGGAGCGGCA\nTTGACAGCGGCTGGGAAGCCAGCCATGCGCCCCTCTCTCTCTCTCTCCCCCCACATAGGACTGTGTGTGT\nGTGCAGAGAAGCCAGTACAAAGGGAGAGGAAGAAAATGGAGGAAAAAGGCCGCCAGGGATGCCGCCGCCC\nTGCCCTGCTG

In [26]:
arctic_lamprey = nuccore_proteins[3]  # KE993814
print(nuccore_proteins)
get_sequence_and_count(protein_df, arctic_lamprey)
# nucleotide sequence (1564372 nucleotides) for Lethenteron camtschaticum (correct)
# Should have 196aa

['GL477576', 'CT004140', 'BAHO01035973', 'KE993814', 'NW_003943621']


(1564372,
 '>KE993814.1 Lethenteron camtschaticum unplaced genomic scaffold scaffold00143, whole genome shotgun sequence\...\nCCCCAAATGAAA\n\n')

In [27]:
squirrel_monkey = nuccore_proteins[4]  # NW_003943621
print(nuccore_proteins)
cnt = get_sequence_and_count(protein_df, squirrel_monkey)[0]
print(cnt)
# nucleotide sequence (30262601 nucleotides) for Saimiri boliviensis (correct)
# Should have 1122 aa

['GL477576', 'CT004140', 'BAHO01035973', 'KE993814', 'NW_003943621']
30262601


In [35]:
# Need to remove proteins whose sequence lengths do not match the aa counts in the input file
# These are either:
# nucleotide sequences for correct protein (GL477576, BAHO01035973, KE993814, NW_003943621)
# mRNA sequence for incorrect protein (CT004140)

# First Remove rows containing NaN's:
protein_df_filt = protein_df.dropna()
protein_df_filt.head()

aa_mismatch_proteins = []
input_aa_cnts = []
api_ret_aa_cnts = []
for i in range(len(protein_df_filt)):
    protein = protein_df_filt.iloc[i]["accession_num"]
    in_cnt = protein_df_filt.iloc[i]["aa_cnt"]  # aa count from infile
    ret_cnt = get_sequence_and_count(protein_df_filt, protein)[0]  ## aa count from API returned sequence
    if in_cnt != ret_cnt:
        aa_mismatch_proteins.append(protein)
        input_aa_cnts.append(in_cnt)
        api_ret_aa_cnts.append(ret_cnt)

aa_mismatch_df = pd.DataFrame({"accession_num": aa_mismatch_proteins,
                               "input_aa_count": input_aa_cnts,
                               "api_returned_aa_count": api_ret_aa_cnts})
aa_mismatch_df

,accession_num,input_aa_count,api_returned_aa_count
0,XP_012352933,732,776
1,XP_008995083,772,497
2,XP_010328565,748,852
3,XP_005983781,758,778
4,XP_013845380,784,451
...,...,...,...
57,XP_418480,1079,1080
58,XP_008116759,1176,2539
59,NP_001119982,1164,93
60,XP_006634278,629,643


In [ ]:
# Need to remove the above five nuccore sequences
# These are either:
# nucleotide sequences for correct protein (GL477576, BAHO01035973, KE993814, NW_003943621)
# mRNA sequence for incorrect protein (CT004140)

# Remove rows containing NaN's
protein_df_filt = protein_df.dropna()
# Remove nuccore proteins
for protein in nuccore_proteins:
    protein_df_filt = protein_df_filt[protein_df_filt["Protein_Accession_Number"] != protein]
protein_df_filt.head()

In [ ]:
# Check nuccore sequences were removed
for protein in nuccore_proteins:
    print(protein_df_filt[protein_df_filt["Protein_Accession_Number"] == protein])

In [ ]:
# Write cleaned sequence series to file
protein_fasta_file = "proteins.faa"
with open(protein_fasta_file, "w") as f:
    for sequence in protein_df_filt["Sequence"]:
        f.write(sequence[:-1])

In [ ]:
# MAPT proteins only 
# still removing nuccore proteins (GL477576, CT004140)
# and those where sequences not found (scaffold11486, JL1528)
infile = os.path.join("Resources", "mapt_only.txt")
with open(infile, "r") as f:
    lines = f.readlines()
mapt_only = [line.replace("\n", "") for line in lines]
len(mapt_only)

In [ ]:
is_mapt = protein_df["Protein_Accession_Number"].isin(mapt_only)
mapt_df = protein_df[is_mapt]
mapt_df

In [ ]:
# Write mapt sequence series to file
mapt_fasta_file = "mapt_only.faa"
with open(mapt_fasta_file, "w") as f:
    for sequence in mapt_df["Sequence"]:
        f.write(sequence[:-1])